# Testing for E2 - using all spindles, training with all data
fistly define the UUID of the experiment to use

In [1]:
experimentId="c9524f21-f587-421e-bfc2-23e64ea50911"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load mass

In [3]:
samplerate=200  #Should rethink this

In [4]:
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
#consider to include this in a function
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.type=='spindle']
annotations=annotations[annotations.duration>minDuration]
annotations=annotations[annotations.duration<maxDuration]
annotations=annotations.reset_index(drop=True)

In [6]:
#check with EDA results (not needed)
print(1-len(annotations)/33458)

0.005648873214178951


In [7]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [8]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## Load experiment results

In [9]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [10]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0001, 0002, 0014]",eb21a368-ea1e-437e-8bc9-5f3b7c717a5b,0.016894
1,0001,E1,[0001],"[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0006, 0010, 0009]",a025f441-a68d-43d3-a521-fb35438037fd,0.018934
2,0001,E1,[0001],"[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0011, 0017, 0018]",7596519c-99e5-46df-bc33-f2a92c47d406,0.017757
3,0001,E1,[0001],"[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0003, 0007, 0019]",c80c584e-d2fc-45cf-9afd-bab63dd6c4cc,0.020286
4,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0005, 0012, 0013]",bfdf80a2-d149-431c-a1d0-dc0e585f5dc9,0.019836
5,0002,E2,[0002],"[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0001, 0002, 0014]",7436536a-3527-4834-a44b-366fcbe0cd99,0.052974
6,0002,E2,[0002],"[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0006, 0010, 0009]",5646178b-299e-43f6-a02e-3cba8c85f86a,0.058345
7,0002,E2,[0002],"[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0011, 0017, 0018]",1ad1098f-696e-423b-b78c-f88499161b60,0.058463
8,0002,E2,[0002],"[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0003, 0007, 0019]",3986159f-0cca-4e15-bbde-8620684cc08f,0.063501
9,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0005, 0012, 0013]",caaad152-e392-4ac5-b420-f736af46b943,0.062113


In [11]:
#we show the difference in class inbalance for the annotation criteria considered
experimentModels[['criteriumName','spindleTimeRate']].groupby('criteriumName').describe()

spindleTimeRate                                          \
                        count      mean       std       min       25%   
criteriumName                                                           
E1                        5.0  0.018741  0.001414  0.016894  0.017757   
E2                        5.0  0.059079  0.004091  0.052974  0.058345   

                                             
                    50%       75%       max  
criteriumName                                
E1             0.018934  0.019836  0.020286  
E2             0.058463  0.062113  0.063501

In [12]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Hyperparameter definition
this should come from a previous evaluation notebook

In [13]:
hyperClose=0.1
hyperDuration=minDuration
hyperThres=0.3
hyperDepth=40

## Testing with E2 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [14]:
experimentModels=experimentModels[experimentModels.criteriumName=='E2'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0002,E2,[0002],"[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0001, 0002, 0014]",7436536a-3527-4834-a44b-366fcbe0cd99,0.052974
1,0002,E2,[0002],"[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0006, 0010, 0009]",5646178b-299e-43f6-a02e-3cba8c85f86a,0.058345
2,0002,E2,[0002],"[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0011, 0017, 0018]",1ad1098f-696e-423b-b78c-f88499161b60,0.058463
3,0002,E2,[0002],"[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0003, 0007, 0019]",3986159f-0cca-4e15-bbde-8620684cc08f,0.063501
4,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0005, 0012, 0013]",caaad152-e392-4ac5-b420-f736af46b943,0.062113


In [15]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix(testSubjectId,featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector(testSubjectId,usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 5


*************************
2 of 5
*************************
3 of 5
*************************
4 of 5
*************************
5 of 5


In [16]:
dumpPickle('experimentModelsTest_E2_temp.pkl',experimentModels)

In [17]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'spindleTimeRate', 'rawF1', 'rawPrecision', 'rawRecall',
       'f1', 'precision', 'recall', 'eventF1', 'eventPrecision',
       'eventRecall'],
      dtype='object')

In [21]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,rawF1,rawPrecision,rawRecall,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0002,E2,[0002],"[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0001, 0002, 0014]",7436536a-3527-4834-a44b-366fcbe0cd99,0.052974,0.687389,0.653175,0.725385,0.688329,0.648444,0.733441,0.743963,0.711558,0.779346
1,0002,E2,[0002],"[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0006, 0010, 0009]",5646178b-299e-43f6-a02e-3cba8c85f86a,0.058345,0.679153,0.743848,0.624812,0.683937,0.742727,0.633770,0.740188,0.764220,0.717698
2,0002,E2,[0002],"[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0011, 0017, 0018]",1ad1098f-696e-423b-b78c-f88499161b60,0.058463,0.695373,0.625366,0.783030,0.695883,0.619804,0.793253,0.760246,0.677141,0.866346
3,0002,E2,[0002],"[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0003, 0007, 0019]",3986159f-0cca-4e15-bbde-8620684cc08f,0.063501,0.661797,0.557096,0.814960,0.661233,0.551666,0.825108,0.698921,0.591555,0.853078
4,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0005, 0012, 0013]",caaad152-e392-4ac5-b420-f736af46b943,0.062113,0.664452,0.570709,0.795042,0.666258,0.566597,0.808461,0.705332,0.581447,0.896015


In [22]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='modelId',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [23]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='modelId',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [24]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='modelId',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [25]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='modelId',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [27]:
experimentModels[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId').describe(percentiles=[0.5])

eventF1                          \
                                       count      mean std       min   
modelId                                                                
1ad1098f-696e-423b-b78c-f88499161b60     1.0  0.760246 NaN  0.760246   
3986159f-0cca-4e15-bbde-8620684cc08f     1.0  0.698921 NaN  0.698921   
5646178b-299e-43f6-a02e-3cba8c85f86a     1.0  0.740188 NaN  0.740188   
7436536a-3527-4834-a44b-366fcbe0cd99     1.0  0.743963 NaN  0.743963   
caaad152-e392-4ac5-b420-f736af46b943     1.0  0.705332 NaN  0.705332   

                                                         eventPrecision  \
                                           50%       max          count   
modelId                                                                   
1ad1098f-696e-423b-b78c-f88499161b60  0.760246  0.760246            1.0   
3986159f-0cca-4e15-bbde-8620684cc08f  0.698921  0.698921            1.0   
5646178b-299e-43f6-a02e-3cba8c85f86a  0.740188  0.740188            1.0   
7436536a-3527-4834-a44b-366fcbe0cd99  0.743963  0.743963            1.0   
caaad152-e392-4ac5-b420-f736af46b943  0.705332  0.705332            1.0   

                                                                        \
                                          mean std       min       50%   
modelId                                                                  
1ad1098f-696e-423b-b78c-f88499161b60  0.677141 NaN  0.677141  0.677141   
3986159f-0cca-4e15-bbde-8620684cc08f  0.591555 NaN  0.591555  0.591555   
5646178b-299e-43f6-a02e-3cba8c85f86a  0.764220 NaN  0.764220  0.764220   
7436536a-3527-4834-a44b-366fcbe0cd99  0.711558 NaN  0.711558  0.711558   
caaad152-e392-4ac5-b420-f736af46b943  0.581447 NaN  0.581447  0.581447   

                                               eventRecall                \
                                           max       count      mean std   
modelId                                                                    
1ad1098f-696e-423b-b78c-f88499161b60  0.677141         1.0  0.866346 NaN   
3986159f-0cca-4e15-bbde-8620684cc08f  0.591555         1.0  0.853078 NaN   
5646178b-299e-43f6-a02e-3cba8c85f86a  0.764220         1.0  0.717698 NaN   
7436536a-3527-4834-a44b-366fcbe0cd99  0.711558         1.0  0.779346 NaN   
caaad152-e392-4ac5-b420-f736af46b943  0.581447         1.0  0.896015 NaN   

                                                                    
                                           min       50%       max  
modelId                                                             
1ad1098f-696e-423b-b78c-f88499161b60  0.866346  0.866346  0.866346  
3986159f-0cca-4e15-bbde-8620684cc08f  0.853078  0.853078  0.853078  
5646178b-299e-43f6-a02e-3cba8c85f86a  0.717698  0.717698  0.717698  
7436536a-3527-4834-a44b-366fcbe0cd99  0.779346  0.779346  0.779346  
caaad152-e392-4ac5-b420-f736af46b943  0.896015  0.896015  0.896015

In [28]:
experimentModels[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean()

,modelId,eventF1,eventPrecision,eventRecall
0,1ad1098f-696e-423b-b78c-f88499161b60,0.760246,0.677141,0.866346
1,3986159f-0cca-4e15-bbde-8620684cc08f,0.698921,0.591555,0.853078
2,5646178b-299e-43f6-a02e-3cba8c85f86a,0.740188,0.764220,0.717698
3,7436536a-3527-4834-a44b-366fcbe0cd99,0.743963,0.711558,0.779346
4,caaad152-e392-4ac5-b420-f736af46b943,0.705332,0.581447,0.896015


In [29]:
experimentModels[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean().mean()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_13816\3541183823.py:1: FutureWarning:

The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.729730
eventPrecision    0.665184
eventRecall       0.822496
dtype: float64

In [30]:
experimentModels[['modelId','eventF1','eventPrecision','eventRecall']].groupby('modelId',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_13816\376514559.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.026399
eventPrecision    0.078318
eventRecall       0.072642
dtype: float64

In [31]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [32]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

## Restrict to only N2 stage

In [33]:
experimentModels_N2=experimentModels.copy()
experimentModels_N2.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate,rawF1,rawPrecision,rawRecall,f1,precision,recall,eventF1,eventPrecision,eventRecall
0,0002,E2,[0002],"[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0003, 0005, 0006, 0007, 0009, 0010, 0011, 001...","[0001, 0002, 0014]",7436536a-3527-4834-a44b-366fcbe0cd99,0.052974,0.687389,0.653175,0.725385,0.688329,0.648444,0.733441,0.743963,0.711558,0.779346
1,0002,E2,[0002],"[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0001, 0002, 0003, 0005, 0007, 0011, 0012, 001...","[0006, 0010, 0009]",5646178b-299e-43f6-a02e-3cba8c85f86a,0.058345,0.679153,0.743848,0.624812,0.683937,0.742727,0.633770,0.740188,0.764220,0.717698
2,0002,E2,[0002],"[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0001, 0002, 0003, 0005, 0006, 0007, 0009, 001...","[0011, 0017, 0018]",1ad1098f-696e-423b-b78c-f88499161b60,0.058463,0.695373,0.625366,0.783030,0.695883,0.619804,0.793253,0.760246,0.677141,0.866346
3,0002,E2,[0002],"[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0001, 0002, 0005, 0006, 0009, 0010, 0011, 001...","[0003, 0007, 0019]",3986159f-0cca-4e15-bbde-8620684cc08f,0.063501,0.661797,0.557096,0.814960,0.661233,0.551666,0.825108,0.698921,0.591555,0.853078
4,0002,E2,[0002],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0005, 0012, 0013]",caaad152-e392-4ac5-b420-f736af46b943,0.062113,0.664452,0.570709,0.795042,0.666258,0.566597,0.808461,0.705332,0.581447,0.896015


In [34]:
annotationsN2=pd.read_csv(datapath+'/MASS/annotations/spindlesFilteredN2.csv')
annotationsN2['samplerate']=samplerate
annotationsN2['subjectId']=annotationsN2.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
annotationsN2['labelerId']=annotationsN2.apply(
    lambda row: str(row.labelerId).zfill(4),axis=1)
annotationsN2['stopTime']=annotationsN2.apply(
    lambda row: row.startTime+row.duration , axis=1)
annotationsN2['startInd']=annotationsN2.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
annotationsN2['stopInd']=annotationsN2.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)

In [35]:
## load hypnograms
stagesAnnotations=pd.read_csv(datapath+"\MASS\stages\stages.csv")
stagesAnnotations['samplerate']=samplerate
stagesAnnotations['subjectId']=stagesAnnotations.apply(
    lambda row: str(row.subjectId).zfill(4),axis=1)
stagesAnnotations['stopTime']=stagesAnnotations.apply(
    lambda row: row.startTime+row.duration , axis=1)
stagesAnnotations['startInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.startTime,row.samplerate) , axis=1)
stagesAnnotations['stopInd']=stagesAnnotations.apply(
    lambda row: seconds2index(row.stopTime,row.samplerate) , axis=1)
stagesAnnotations.head(5)

hypnograms={}
for ind, row in signalsMetadata.iterrows():
    subjectId=row.subjectId
    thisStages=stagesAnnotations[stagesAnnotations.subjectId==subjectId]
    excerptDimension=int(row.duration*row.samplerate)
    thisHypnogram=np.ones((excerptDimension,))*np.nan
    for ind_stg, row_stg in thisStages.iterrows():
        thisHypnogram[row_stg.startInd:row_stg.stopInd]=row_stg.value
    hypnograms[subjectId]=thisHypnogram   

In [36]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels_N2.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels_N2)) )
    #load model
    model=loadBooster(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    testHypnogram=hypnograms[testSubjectId]
    testN2Mask=testHypnogram==2
    #Define annotations criterium
    usedAnnotations=annotationsN2[annotationsN2.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    testFeatures=loadFeatureMatrix(testSubjectId,featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector(testSubjectId,usedAnnotations,signalsMetadata,samplerate)

    #Predict
    testDMatrix=xgb.DMatrix(data=testFeatures)
    probabilities=model.predict(testDMatrix,iteration_range=(0,hyperDepth))
    rawLabels=probabilities>=hyperThres

    #Apply mask ---------------------------------->
    rawLabels=rawLabels*testN2Mask
    #<---------------------------------------------

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<maxDuration]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)

    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels_N2['rawF1']=rawF1s
experimentModels_N2['rawPrecision']=rawPrecisions
experimentModels_N2['rawRecall']=rawRecalls

experimentModels_N2['f1']=f1s
experimentModels_N2['precision']=precisions
experimentModels_N2['recall']=recalls

experimentModels_N2['eventF1']=eventF1s
experimentModels_N2['eventPrecision']=eventPrecisions
experimentModels_N2['eventRecall']=eventRecalls


*************************
1 of 5


TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
dumpPickle('experimentModelsTest_N2_E2_temp.pkl',experimentModels_N2)

In [ ]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [ ]:
fig=px.scatter(experimentModels_N2,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['rawF1'], y=experimentModels_N2['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [ ]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [ ]:
fig=px.scatter(experimentModels_N2,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [ ]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0001     1.0  0.716014 NaN  0.716014  0.716014  0.716014            1.0   
0002     1.0  0.866606 NaN  0.866606  0.866606  0.866606            1.0   
0003     1.0  0.788018 NaN  0.788018  0.788018  0.788018            1.0   
0005     1.0  0.747412 NaN  0.747412  0.747412  0.747412            1.0   
0006     1.0  0.673252 NaN  0.673252  0.673252  0.673252            1.0   
0007     1.0  0.784012 NaN  0.784012  0.784012  0.784012            1.0   
0009     1.0  0.738625 NaN  0.738625  0.738625  0.738625            1.0   
0010     1.0  0.861827 NaN  0.861827  0.861827  0.861827            1.0   
0011     1.0  0.800000 NaN  0.800000  0.800000  0.800000            1.0   
0012     1.0  0.824255 NaN  0.824255  0.824255  0.824255            1.0   
0013     1.0  0.774992 NaN  0.774992  0.774992  0.774992            1.0   
0014     1.0  0.713333 NaN  0.713333  0.713333  0.713333            1.0   
0017     1.0  0.798624 NaN  0.798624  0.798624  0.798624            1.0   
0018     1.0  0.796327 NaN  0.796327  0.796327  0.796327            1.0   
0019     1.0  0.723936 NaN  0.723936  0.723936  0.723936            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0001  0.943614 NaN  0.943614  0.943614  0.943614         1.0  0.577114 NaN   
0002  0.833407 NaN  0.833407  0.833407  0.833407         1.0  0.902520 NaN   
0003  0.869919 NaN  0.869919  0.869919  0.869919         1.0  0.720067 NaN   
0005  0.629220 NaN  0.629220  0.629220  0.629220         1.0  0.919492 NaN   
0006  0.893145 NaN  0.893145  0.893145  0.893145         1.0  0.540244 NaN   
0007  0.737322 NaN  0.737322  0.737322  0.737322         1.0  0.836538 NaN   
0009  0.779442 NaN  0.779442  0.779442  0.779442         1.0  0.701840 NaN   
0010  0.926888 NaN  0.926888  0.926888  0.926888         1.0  0.805541 NaN   
0011  0.716795 NaN  0.716795  0.716795  0.716795         1.0  0.904336 NaN   
0012  0.808528 NaN  0.808528  0.808528  0.808528         1.0  0.840555 NaN   
0013  0.669882 NaN  0.669882  0.669882  0.669882         1.0  0.918681 NaN   
0014  0.583502 NaN  0.583502  0.583502  0.583502         1.0  0.916931 NaN   
0017  0.868263 NaN  0.868263  0.868263  0.868263         1.0  0.739389 NaN   
0018  0.725137 NaN  0.725137  0.725137  0.725137         1.0  0.882817 NaN   
0019  0.598706 NaN  0.598706  0.598706  0.598706         1.0  0.914370 NaN   

                                    
           min       50%       max  
test                                
0001  0.577114  0.577114  0.577114  
0002  0.902520  0.902520  0.902520  
0003  0.720067  0.720067  0.720067  
0005  0.919492  0.919492  0.919492  
0006  0.540244  0.540244  0.540244  
0007  0.836538  0.836538  0.836538  
0009  0.701840  0.701840  0.701840  
0010  0.805541  0.805541  0.805541  
0011  0.904336  0.904336  0.904336  
0012  0.840555  0.840555  0.840555  
0013  0.918681  0.918681  0.918681  
0014  0.916931  0.916931  0.916931  
0017  0.739389  0.739389  0.739389  
0018  0.882817  0.882817  0.882817  
0019  0.914370  0.914370  0.914370

In [ ]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0001,0.716014,0.943614,0.577114
1,0002,0.866606,0.833407,0.902520
2,0003,0.788018,0.869919,0.720067
3,0005,0.747412,0.629220,0.919492
4,0006,0.673252,0.893145,0.540244
5,0007,0.784012,0.737322,0.836538
6,0009,0.738625,0.779442,0.701840
7,0010,0.861827,0.926888,0.805541
8,0011,0.800000,0.716795,0.904336
9,0012,0.824255,0.808528,0.840555


In [ ]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              6.668000e+54
eventF1           7.738157e-01
eventPrecision    7.722513e-01
eventRecall       8.080291e-01
dtype: float64

In [ ]:
experimentModels_N2[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_4160\813125388.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.055130
eventPrecision    0.118006
eventRecall       0.126000
dtype: float64

In [ ]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels_N2.eventPrecision,
    'event F1':experimentModels_N2.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels_N2.eventRecall,
    'event F1':experimentModels_N2.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [ ]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels_N2['eventF1'], y=experimentModels_N2['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### remove outliers

In [ ]:
np.setdiff1d(experimentModels_N2.test,['0006'])

array(['0001', '0002', '0003', '0005', '0007', '0009', '0010', '0011',
       '0012', '0013', '0014', '0017', '0018', '0019'], dtype=object)

In [ ]:
experimentModels_N2[experimentModels_N2.test.isin(np.setdiff1d(experimentModels_N2.test,['0001','0014','0019']))][['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              1.666917e+43
eventF1           7.878293e-01
eventPrecision    7.881624e-01
eventRecall       8.093351e-01
dtype: float64

In [ ]:
annotationPairToGraph(gtAnnotations,processedAnnotations)

NameError: name 'px' is not defined